In [2]:
import requests
import pandas as pd
import json

API_KEY = 'yC64acO6xj53iW03VTdPX6QZxY5Y7najMElljT50'
HEADERS = {'x-api-key': API_KEY}


In [3]:
# Load character mapping
with open('character_mapping.json', 'r', encoding='utf-8') as f:
    character_map = json.load(f)

# Load weapon mapping
with open('weapon_mapping.json', 'r', encoding='utf-8') as f:
    weapon_map = json.load(f)

with open('item_mapping.json', 'r', encoding='utf-8') as f:
    item_map = json.load(f)
    
with open('augment_mapping.json', 'r', encoding='utf-8') as f:
    augment_map = json.load(f)


In [ ]:
game_id = 46678263

# Function to get the game data for all players in a game
def get_game_data(game_id):
    url = f'https://open-api.bser.io/v1/games/{game_id}'
    response = requests.get(url, headers=HEADERS)

    if response.status_code != 200:
        print(f"Failed to fetch game data: {response.status_code} - {response.text}")
        return None

    data = response.json()

    # Initialize list to store player data
    players_data = []

    # Define valid weapon mastery IDs (as strings)
    weapon_mastery_ids = {
        "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11",
        "12", "13", "14", "15", "16", "17", "18", "19", "20",
        "21", "22", "23", "24", "25"
    }
        
    # Iterate through players in the 'userGames' list
    for player in data.get('userGames', []):
        user_num = player.get('userNum', '')
        nickname = player.get('nickname', '')
        game_id = player.get('gameId', '')
        season_id = player.get('seasonId', '')
        game_Rank = player.get('gameRank', '')
        character_Num = player.get('characterNum', '')
        team_kill = player.get('teamKill','')
        kill = player.get('playerKill', '')
        death = player.get('playerDeaths', '')
        assist = player.get('playerAssistant','')
        dmg_player = player.get('damageToPlayer','')
        dmg_monster = player.get('damageToMonster','')
        

        # Get masteryLevel dict
        mastery = player.get('masteryLevel', {})
        
        # Extract weapon mastery ID
        weapon_type_id = ''
        for mastery_id in mastery:
            if mastery_id in weapon_mastery_ids:
                weapon_type_id = mastery_id  # Keep as string
                break

        # Look up readable names
        character_name = character_map.get(str(character_Num), f"Unknown({character_Num})")
        weapon_name = weapon_map.get(weapon_type_id, f"Unknown({weapon_type_id})")
        
        # Get equipment data
        equipment = player.get('equipment', {})
        item_weapon = item_map.get(str(equipment.get('0', '')), 'Unknown(Weapon)')
        item_chest = item_map.get(str(equipment.get('1', '')), 'Unknown(Chest)')
        item_head = item_map.get(str(equipment.get('2', '')), 'Unknown(Head)')
        item_arm = item_map.get(str(equipment.get('3', '')), 'Unknown(Arm)')
        item_leg = item_map.get(str(equipment.get('4', '')), 'Unknown(Leg)')
        
        # === AUGMENT PARSING ===
        first_sub = player.get('traitFirstSub', [])
        second_sub = player.get('traitSecondSub', [])

        trait_main1 = augment_map.get(str(first_sub[0]), f"Unknown({first_sub[0]})") if len(first_sub) > 0 else ""
        trait_main2 = augment_map.get(str(first_sub[1]), f"Unknown({first_sub[1]})") if len(first_sub) > 1 else ""
        trait_sub1 = augment_map.get(str(second_sub[0]), f"Unknown({second_sub[0]})") if len(second_sub) > 0 else ""
        trait_sub2 = augment_map.get(str(second_sub[1]), f"Unknown({second_sub[1]})") if len(second_sub) > 1 else ""

        # Append player data to the list
        players_data.append({
            'userNum': user_num,
            'nickname': nickname,
            'gameId': game_id,
            'seasonId': season_id,
            'gameRank': game_Rank,
            'character': character_name,
            'weapon': weapon_name,
            'itemWeapon': item_weapon,
            'itemChest': item_chest,
            'itemHead': item_head,
            'itemArm': item_arm,
            'itemLeg': item_leg,
            'Trait_Main1': trait_main1,
            'Trait_Main2': trait_main2,
            'Trait_Sub1': trait_sub1,
            'Trait_Sub2': trait_sub2,
            'TeamKill' : team_kill,
            'Kill' : kill,
            'Death' : death,
            'Assist' : assist,
            'Dmg_Player' : dmg_player,
            'Dmg_Monster' : dmg_monster
        })

    return players_data

{'userNum': 5359969, 'nickname': '尋風追憶', 'gameId': 46678263, 'seasonId': 31, 'gameRank': 7, 'character': 'Leon', 'weapon': 'Glove', 'itemWeapon': 'Obsidian_Jitte', 'itemChest': 'Covert_Agent_Uniform', 'itemHead': "Astronaut's_Helmet", 'itemArm': "Pharaoh's_Artifact", 'itemLeg': 'Delta_Red', 'Trait_Main1': 'Open_Wounds', 'Trait_Main2': 'Overwatch', 'Trait_Sub1': 'Dismantle_Goliath', 'Trait_Sub2': 'Anima_Reaper', 'TeamKill': 12, 'Kill': 3, 'Death': 2, 'Assist': 7, 'Dmg_Player': 10909, 'Dmg_Monster': 31758}
{'userNum': 5294646, 'nickname': '뎁미니스트', 'gameId': 46678263, 'seasonId': 31, 'gameRank': 5, 'character': 'Debi_&_Marlene', 'weapon': 'Two-handed_Sword', 'itemWeapon': 'Aurora_Longsword', 'itemChest': 'Phantom_Jacket', 'itemHead': 'Commander_Headset', 'itemArm': 'Moonlight_Pendant', 'itemLeg': 'Cowboy_Boots', 'Trait_Main1': 'Camping_Guide', 'Trait_Main2': 'Penny_Pincher', 'Trait_Sub1': 'Embolden', 'Trait_Sub2': "Dine_n'_Dash", 'TeamKill': 15, 'Kill': 2, 'Death': 1, 'Assist': 11, 'Dmg_P

In [7]:
import random
import time


# Function to run the process multiple times with random game IDs within a range
def run_multiple_games(start_game_id, end_game_id, num_runs, mode='append'):
    for _ in range(num_runs):
        # Generate a random game ID within the specified range
        game_id = random.randint(start_game_id, end_game_id)
        
        # Get game data
        players_data = get_game_data(game_id)
        
        # Save to CSV
        if players_data:
            save_players_data_to_csv(players_data, mode=mode)
        
        # Optionally, add a delay between requests to avoid overloading the API or getting blocked
        time.sleep(1)  # Delay in seconds, can be adjusted

# Example usage
run_multiple_games(start_game_id=46678000, end_game_id=46679000, num_runs=3, mode='append')


NameError: name 'get_game_data' is not defined

In [39]:
print(len(players_data))


24


In [64]:
import csv
import os

def save_players_data_to_csv(players_data, output_file='players_data.csv', mode='append'):
    if not players_data:
        print("No player data to save.")
        return

    headers = players_data[0].keys()

    if mode == 'replace':
        # Replace mode: overwrite the file
        file_exists = False
    elif mode == 'append':
        # Append mode: check if the file exists
        file_exists = os.path.isfile(output_file)
    else:
        print("Invalid mode! Use 'append' or 'replace'.")
        return

    with open(output_file, 'a' if mode == 'append' else 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=headers)
        if not file_exists:
            writer.writeheader()  # Write headers only if the file is being created
        writer.writerows(players_data)

    print(f"Data saved to {output_file} in {mode} mode.")


In [ ]:
save_players_data_to_csv(players_data, mode='append')


Data saved to players_data.csv in append mode.


In [53]:
save_players_data_to_csv(players_data, mode='replace')


Data saved to players_data.csv in replace mode.


In [4]:
import random
import time

# Function to run the process multiple times with random game IDs within a range
def run_multiple_games(start_game_id, end_game_id, num_runs, mode='append'):
    for _ in range(num_runs):
        # Generate a random game ID within the specified range
        game_id = random.randint(start_game_id, end_game_id)
        
        # Get game data
        players_data = get_game_data(game_id)
        
        # Save to CSV
        if players_data:
            save_players_data_to_csv(players_data, mode=mode)
        
        # Optionally, add a delay between requests to avoid overloading the API or getting blocked
        time.sleep(1)  # Delay in seconds, can be adjusted

# Example usage
run_multiple_games(start_game_id=46678000, end_game_id=46679000, num_runs=3, mode='append')


NameError: name 'get_game_data' is not defined